In [51]:
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue, deque
from copy import  copy
from math import ceil 
import numpy as np
from tqdm.auto import tqdm
import time 

PROBLEM_SIZE  = 10 #elements to cover
NUM_SETS = 20
SETS =  tuple(np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])  for _ in range(NUM_SETS) )
    #the value True means that the set contains the element
    #we randomly create NUM_SETS sets of PROBLEM_SIZE elements (True/False)\
SETS

(array([False, False, False,  True, False,  True, False, False, False,
         True]),
 array([False,  True, False, False,  True,  True,  True, False,  True,
        False]),
 array([ True, False, False, False, False, False, False, False,  True,
        False]),
 array([False, False, False, False, False, False,  True, False, False,
        False]),
 array([False, False,  True, False, False, False, False, False, False,
        False]),
 array([False,  True, False,  True,  True, False,  True, False, False,
         True]),
 array([ True, False, False, False, False, False, False, False,  True,
        False]),
 array([False,  True, False,  True, False, False,  True, False,  True,
        False]),
 array([ True, False, False,  True, False, False,  True,  True,  True,
        False]),
 array([ True, False, False, False,  True, False,  True, False, False,
        False]),
 array([ True,  True, False, False,  True, False,  True, False, False,
         True]),
 array([False, False, False, Fal

In [52]:
class Individual() :
    def __init__(self, current_solution : list, fitness : callable):
        self.current_solution = current_solution
        self.covering = [sum(SETS[val]) for val in current_solution]
        self.fitness = fitness
        self.fitness_value = fitness(current_solution)

    def __str__(self):
        return f'{[value for value in self.current_solution]} with fitness : {self.fitness_value} and covering sets :{self.covering}'    

def fitness(current_solution : list) -> int:
    return len(current_solution)

def covered(state : list) -> int:
    return reduce(
        np.logical_or,
        [SETS[i] for i in state],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def goal_check(state):
    return np.all(covered(state)) 

def create_random_individual(num_individuals : int) -> list[Individual]:  #initialize the population 
    count = 0
    population = []
    while count < num_individuals:
        solution = list(set([randint(0, NUM_SETS - 1) for _ in range(randint(0, NUM_SETS - 1))]))
        if goal_check(solution):

            indiv = Individual(solution, fitness)
            population.append(indiv)
            count += 1
    return population


####create a population of individuals 
population = create_random_individual(10) 
for i in range(len(population)):
    print(population[i]) 
    print(goal_check(population[i].current_solution))     


[0, 1, 2, 5, 7, 8, 11, 12, 13, 17] with fitness : 10 and covering sets :[3, 5, 2, 5, 4, 5, 1, 3, 3, 6]
True
[4, 5, 8, 9, 11, 14] with fitness : 6 and covering sets :[1, 5, 5, 3, 1, 4]
True
[0, 1, 4, 5, 7, 11, 12, 14, 18, 19] with fitness : 10 and covering sets :[3, 5, 1, 5, 4, 1, 3, 4, 1, 2]
True
[2, 6, 7, 8, 13, 14, 15, 17, 18] with fitness : 9 and covering sets :[2, 2, 4, 5, 3, 4, 2, 6, 1]
True
[0, 1, 2, 3, 7, 10, 11, 16, 17, 19] with fitness : 10 and covering sets :[3, 5, 2, 1, 4, 5, 1, 4, 6, 2]
True
[0, 6, 7, 9, 10, 11, 13, 17, 18] with fitness : 9 and covering sets :[3, 2, 4, 3, 5, 1, 3, 6, 1]
True
[3, 4, 6, 7, 8, 9, 10, 12, 13, 18] with fitness : 10 and covering sets :[1, 1, 2, 4, 5, 3, 5, 3, 3, 1]
True
[2, 4, 6, 7, 10, 11, 12, 15, 18] with fitness : 9 and covering sets :[2, 1, 2, 4, 5, 1, 3, 2, 1]
True
[0, 2, 3, 8, 9, 10, 12, 13, 14, 17, 19] with fitness : 11 and covering sets :[3, 2, 1, 5, 3, 5, 3, 3, 4, 6, 2]
True
[0, 1, 3, 5, 6, 7, 8, 9, 10, 14, 17] with fitness : 11 and cove

In [75]:
class Evolutionary_algorithm():
    def selected_population(self, population : list[Individual]) -> list[Individual]:
        pass

    def crossover(self, population : list[Individual]) -> list[Individual]:
        return population

    def mutation(self, population : list[Individual], mutation_rate : float = 0.1) -> list[Individual]:
        copy_pop = copy(population)
        for i in range(len(copy_pop)):
            if random() < mutation_rate:
                ##first action, we try to mutate the individual by removing the set that covers the minimum elements
                index_to_remove = np.argmin(copy_pop[i].covering)
                val = copy_pop[i].current_solution.pop(index_to_remove)
                valid = goal_check(copy_pop[i].current_solution)
                if valid: 
                    copy_pop[i].covering.pop(index_to_remove)
                    copy_pop[i].fitness_value = copy_pop[i].fitness(copy_pop[i].current_solution)
                else : 
                    copy_pop[i].current_solution.insert(val, index_to_remove)
        return copy_pop
                




    def k_tournament_selection(self, population : list[Individual], k :int = 2) -> list[Individual]:
        selected_parents = []
        ##we select k individuals randomly and we select the best one
        for _ in range(len(population)):
            selected = []
            for index in range(k):  
                selected.append(choice(population))
            selected.sort(key = lambda x : x.fitness_value)
            selected_parents.append(selected[0])
        return selected_parents
            
def search_best(population : list[Individual], current_best : Individual) -> Individual:
    copy_pop = copy(population)
    copy_pop.sort(key = lambda x : x.fitness_value)
    best_solution = copy_pop[0]
    if best_solution.fitness_value < current_best.fitness_value:
        return copy(best_solution)
    else :
        return current_best


##generation process 
def evolutionary_algorithm (generations : int = 1000) :
    time_start = time.time()
    population = create_random_individual(PROBLEM_SIZE * 5)
    for i in population:
        print(i)
    gen = 0
    copy_pop = copy(population)
    copy_pop.sort(key = lambda x : x.fitness_value)
    best_solution = copy_pop[0]
    print(f'Original best solution : {best_solution}') 
    current_best = copy(best_solution)
    evolve = Evolutionary_algorithm()
    while gen < generations:
        selected = evolve.k_tournament_selection(population,3)
        
        offspring = evolve.crossover(selected)

        offspring = evolve.mutation(offspring, mutation_rate = 0.1)

        population = offspring
        current_best = search_best(population, current_best)

        gen += 1
    
    print(f'Current best solution : {current_best}')   
    time_end = time.time()
    print(f'Generation process in {time_end - time_start} seconds')

evolutionary_algorithm()

[0, 1, 2, 4, 7, 8, 11, 12, 14, 16, 17, 18] with fitness : 12 and covering sets :[3, 5, 2, 1, 4, 5, 1, 3, 4, 4, 6, 1]
[0, 1, 4, 7, 8, 9, 10, 12, 13, 15, 19] with fitness : 11 and covering sets :[3, 5, 1, 4, 5, 3, 5, 3, 3, 2, 2]
[0, 5, 7, 8, 12, 15, 17, 18, 19] with fitness : 9 and covering sets :[3, 5, 4, 5, 3, 2, 6, 1, 2]
[1, 3, 4, 5, 6, 7, 8, 11, 12, 17, 19] with fitness : 11 and covering sets :[5, 1, 1, 5, 2, 4, 5, 1, 3, 6, 2]
[0, 1, 2, 3, 6, 7, 10, 11, 12, 15, 16, 17, 18, 19] with fitness : 14 and covering sets :[3, 5, 2, 1, 2, 4, 5, 1, 3, 2, 4, 6, 1, 2]
[1, 4, 5, 6, 7, 8, 11, 13, 14, 15, 17, 18, 19] with fitness : 13 and covering sets :[5, 1, 5, 2, 4, 5, 1, 3, 4, 2, 6, 1, 2]
[1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 14, 17] with fitness : 12 and covering sets :[5, 2, 1, 1, 5, 2, 4, 5, 1, 3, 4, 6]
[0, 1, 2, 3, 5, 6, 7, 8, 10, 13, 17] with fitness : 11 and covering sets :[3, 5, 2, 1, 5, 2, 4, 5, 5, 3, 6]
[0, 2, 6, 9, 10, 11, 13, 14, 15, 16, 17, 18] with fitness : 12 and covering sets :[3, 2, 